# Data Access

> Welcome!
> 
> Please download this notebook and load it into your [PAWS instance](https://hub-paws.wmcloud.org/)

This notebook provides a basic overview of the following methods that are used to access data from Wikidata:

- [Wikidata REST API](https://www.wikidata.org/wiki/Wikidata:REST_API)
- [Wikidata Integrator](https://github.com/SuLab/WikidataIntegrator)
- [SPARQLWrapper](https://github.com/RDFLib/sparqlwrapper)
- [Pywikibot](https://github.com/wikimedia/pywikibot)
- [wikidata](https://github.com/dahlia/wikidata)
- [Wiki Replicas](https://wikitech.wikimedia.org/wiki/Wiki_Replicas)

We'll start with the REST API and then move to Python packages! All examples will be based on getting information about [Douglas Adams](https://en.wikipedia.org/wiki/Douglas_Adams), the author of the [Hitchhiker's Guide to the Galaxy](https://en.wikipedia.org/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy) and [Q42](https://www.wikidata.org/wiki/Q42) in Wikidata. Resources for learning more about the tools are included in each section. We'll also be covering how to access Wikidata's information from [data dumps](https://www.wikidata.org/wiki/Wikidata:Database_download#json_dumps_(recommended)) in a following notebook.

**Note**: the tools are separated into suggested and other tools so you know what would be best to focus on.

**Note**: please see [PAWS examples and recipes](https://wikitech.wikimedia.org/wiki/PAWS/PAWS_examples_and_recipes) for a full list of tutorials on how to use Wikidata's data in PAWS.

## SQL Syntax

SQL and other query languages like SPARQL have never developed a common format that they should be written in. We'd suggest the following for SQL during the program:

- Upper case all statements
  - `SELECT`, `SELECT DISTINCT`, `WHERE`, `ORDER BY`, etc
- Upper case all operators and constant values
  - `AND`, `OR`, `IN`, `NOT IN`, `JOIN`, `LEFT JOIN`, `IS NULL`, etc
- Lower case all functions as the parentheses will make them apparent 
  - `count()`, `sum()`, etc
- After a statement always go to the next line and indent
  - Do the same for `ON` during a join
- Use aliases (`table AS t` or `long_value AS value`) liberally
  - It's a good habit to shorten your query and make sure all elements are detailed

## Imports

In [1]:
# pip install sparqlwrapper

In [2]:
# pip install wikidata

In [3]:
# pip install wikidataintegrator

In [4]:
# pip install jupyter-black

In [5]:
%load_ext jupyter_black

In [6]:
import requests

import pymysql
import pywikibot  # included in all PAWS instances
from SPARQLWrapper import JSON, POST, SPARQLWrapper
from wikidata.client import Client
from wikidataintegrator import wdi_core

## Suggested Tools

### Wikidata REST API

Please see the [Wikidata REST API documentation](https://www.wikidata.org/wiki/Wikidata:REST_API) for more information.

The following is a very simple example of doing get requests with the API, but there's [so much more](https://doc.wikimedia.org/Wikibase/master/js/rest-api/) that can be done with it!

**Note**: in the REST API properties are referred to as statements.

In [7]:
def rest_api_get_request(wd_qid: str, term: str = ""):
    api_endpoint = "https://www.wikidata.org/w/rest.php/wikibase/v0"
    request_string = api_endpoint + "/entities/items/" + f"{wd_qid}"
    if term != "":
        request_string += f"/{term}"

    request = requests.get(request_string)

    return request.json()

In [8]:
rest_api_q42_label = rest_api_get_request(wd_qid="Q42", term="labels")["en"]
rest_api_q42_label

'Douglas Adams'

In [9]:
rest_api_q42_notable_work_ids = [
    r["value"]["content"]
    for r in rest_api_get_request(wd_qid="Q42", term="statements")["P800"]
]

In [10]:
wd_q42_notable_work_labels = [
    rest_api_get_request(wd_qid=i, term="labels")["en"]
    for i in rest_api_q42_notable_work_ids
]

In [11]:
wd_q42_notable_work_labels

["The Hitchhiker's Guide to the Galaxy pentalogy",
 'Dirk Gently series',
 'The Private Life of Genghis Khan']

### Wikidata Integrator

Please see the [project readme on GitHub](https://github.com/SuLab/WikidataIntegrator#wikidata-integrator) for more information.

Note that although Wikidata Integrator can also run SPARQL queries, SPARQLWrapper is the preferred query package as it has fewer limitations.

In [12]:
wdi_q42 = wdi_core.WDItemEngine(wd_item_id="Q42")

In [13]:
wdi_q42_dict = wdi_q42.get_wd_json_representation()

In [14]:
wdi_q42_dict["labels"]["en"]["value"]

'Douglas Adams'

In [15]:
wdi_q42_notable_work_ids = [
    w["mainsnak"]["datavalue"]["value"]["id"] for w in wdi_q42_dict["claims"]["P800"]
]

In [16]:
wdi_q42_notable_work_labels = [
    wdi_core.WDItemEngine(wd_item_id=i).get_wd_json_representation()["labels"]["en"][
        "value"
    ]
    for i in wdi_q42_notable_work_ids
]

In [17]:
wdi_q42_notable_work_labels

["The Hitchhiker's Guide to the Galaxy pentalogy",
 'Dirk Gently series',
 'The Private Life of Genghis Khan']

### SPARQLWrapper

Please see the [How to use section](https://github.com/RDFLib/sparqlwrapper#how-to-use) of the [SPARQLWrapper repository](https://github.com/RDFLib/sparqlwrapper) on GitHub for more information.

We'll give a more thorough overview of the SPARQL query language in the next notebook!

In [18]:
# Set SPARQLWrapper query conditions.
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)
sparql.setMethod(POST)

In [19]:
sw_q42_label_query = """
SELECT
    ?entity
    ?entityLabel

WHERE {
    wd:Q42 rdfs:label ?entityLabel.

    # Use the labelling service to allow us to query `authorLabel`.
    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en".
    }
}
"""

In [20]:
sparql.setQuery(sw_q42_label_query)
sw_q42_label_results = sparql.query().convert()

In [21]:
sw_q42_label_bindings = sw_q42_label_results["results"]["bindings"]

In [22]:
sw_q42_label_bindings_en = [
    r for r in sw_q42_label_bindings if r["entityLabel"]["xml:lang"] == "en"
][0]

In [23]:
sw_q42_label_bindings_en["entityLabel"]["value"]

'Douglas Adams'

In [24]:
sw_q42_notable_works_query = """
SELECT
    ?work
    ?workLabel
    ?workDescription

WHERE
    {
        # Douglas Adams  #author   # notable work
        wd:Q42           wdt:P800   ?work.

        SERVICE wikibase:label {
            bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en".
        }
    }
"""

In [25]:
sparql.setQuery(sw_q42_notable_works_query)
sw_q42_notable_works_results = sparql.query().convert()

In [26]:
sw_q42_notable_works_bindings = sw_q42_notable_works_results["results"]["bindings"]

In [27]:
sw_q42_notable_works_formatted = []
for r in sw_q42_notable_works_bindings:  # query_results is also a list
    r_dict = {k: r[k]["value"] for k in r.keys()}

    sw_q42_notable_works_formatted.append(r_dict)

In [28]:
sw_q42_notable_works_formatted[:3]

[{'work': 'http://www.wikidata.org/entity/Q25169',
  'workLabel': "The Hitchhiker's Guide to the Galaxy pentalogy",
  'workDescription': '1979-1992 series of five books by Douglas Adams'},
 {'work': 'http://www.wikidata.org/entity/Q7758404',
  'workLabel': 'The Private Life of Genghis Khan',
  'workDescription': 'story by Douglas Adams and Graham Chapman'},
 {'work': 'http://www.wikidata.org/entity/Q20736364',
  'workLabel': 'Dirk Gently series',
  'workDescription': 'series of three books by Douglas Adams'}]

## Other Tools

### Pywikibot

Please see the [Wikidata Pywikibot](https://www.wikidata.org/wiki/Wikidata:Pywikibot_-_Python_3_Tutorial) tutorial for more information.

In [29]:
# Normally in Pywikibot the first argument is a language ISO code.
# For Wikidata we just repeat it (or use only one argument).
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()
pwb_q42 = pywikibot.ItemPage(repo, "Q42")

In [30]:
pwb_q42_dict = pwb_q42.get()

In [31]:
pwb_q42_dict.keys()

dict_keys(['labels', 'descriptions', 'aliases', 'claims', 'sitelinks'])

In [32]:
pwb_q42_dict["labels"]["en"]

'Douglas Adams'

In [33]:
# P800 is the Wikidata property for notable works.
pwb_q42_notable_work_qids = [
    "Q"
    + str(
        pwb_q42_dict["claims"]["P800"][i].toJSON()["mainsnak"]["datavalue"]["value"][
            "numeric-id"
        ]
    )
    for i in range(len(pwb_q42_dict["claims"]["P800"]))
]

In [34]:
pwb_q42_notable_work_labels = [
    pywikibot.ItemPage(repo, i).get()["labels"]["en"] for i in pwb_q42_notable_work_qids
]

In [35]:
pwb_q42_notable_work_labels

["The Hitchhiker's Guide to the Galaxy pentalogy",
 'Dirk Gently series',
 'The Private Life of Genghis Khan']

### wikidata

Please see the [docs on readthedocs](https://wikidata.readthedocs.io/en/stable/) for more information.

In [36]:
# Set the client from which we'll access Wikidata.
client = Client()
wd_q42_entity = client.get("Q42", load=True)

In [37]:
wd_q42_entity.label  # below is multilingual text that we can access like a dictionary

m'Douglas Adams'

In [38]:
wd_q42_entity.description["en"]

'English author and humourist (1952–2001)'

In [39]:
wd_q42_notable_work_ids = [
    w["mainsnak"]["datavalue"]["value"]["id"]
    for w in wd_q42_entity.data["claims"]["P800"]
]

In [40]:
wd_q42_notable_work_labels = [
    client.get(i, load=True).label["en"] for i in wd_q42_notable_work_ids
]

In [41]:
wd_q42_notable_work_labels

["The Hitchhiker's Guide to the Galaxy pentalogy",
 'Dirk Gently series',
 'The Private Life of Genghis Khan']

### Wiki Replicas

Please see the [Wiki Replicas documentation page on Wikitech](https://wikitech.wikimedia.org/wiki/Wiki_Replicas) for more information.

Wiki Replicas are databases containing a sanitized version of the Wikimedia's [production databases in our data lake](https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake).

**Note**: as Wikidata is so large, we do not have a `text` (or `pagecontent`) table as in other wikis, so this is not a viable option for working with Wikidata. It's brought up here only to mention it as a possibility when working with other Wikimedia projects.

As a simple example, let's make the needed functions and find out how many items there are on Wikidata ([reference](https://public-paws.wmcloud.org/User:JHernandez_(WMF)/Accessing%20Wikireplicas%20from%20PAWS.ipynb)).

In [42]:
def connect_to_wiki_db(
    wiki: str, replica_type: str = "analytics", path_to_cnf: str = "."
):
    """
    Connects to a host and database of the same name.

    Parameters
    ----------
        wiki: str
            The Wikimedia wiki to connect to.

        replica_type: str (default=analytics)
            The replica type for the connection - either "analytics" or "web".

        path_to_cnf: str (default=analytics)
            The path to the configuration file needed for the database connection.

    Returns
    -------
        A database connection for making MySQL queries.
    """
    assert (
        replica_type == "analytics" or replica_type == "web"
    ), "`replica_type` must be either 'analytics' or 'web'."

    return pymysql.connect(
        host=f"{wiki}.{replica_type}.db.svc.wikimedia.cloud",
        read_default_file=f"{path_to_cnf}/.my.cnf",
        database=f"{wiki}_p",
        charset="utf8",
    )

In [43]:
def query_wiki_db(conn, query, args):
    """
    Execute a SQL query against the connection and return the results.

    Parameters
    ----------
        conn: pymysql.connections.Connection
            The Wikimedia wiki to connect to.

        query: str
            The query to execute against the database.

        args: str or list(str)
            Arguments that will be passed to the query at %s indices.

    Returns
    -------
        A database connection for making MySQL queries.
    """
    with conn.cursor() as cur:
        cur.execute(query, args=args)
        data = cur.fetchall()
        return data

In [44]:
# path_to_cnf points to the PAWS root where the configuration file .my.cnf is that has a username and password.
wikidata_db_conn = connect_to_wiki_db(
    wiki="wikidatawiki", replica_type="analytics", path_to_cnf="../.."
)

The schema for the `page` table below can be found at https://www.mediawiki.org/wiki/Manual:Page_table.

In [45]:
# We have to pass args to %s below and will use 1=1 get all results.
# The WHERE clause also assures we're on the Main namespace (not another like discussions) and we're looking at items only.
total_wd_items_query = """
SELECT
    count(DISTINCT page_title) AS total_qids

FROM
    page

WHERE
    page_namespace = 0
    AND page_content_model = 'wikibase-item'
    AND %s
"""

In [46]:
# This query will take a bit longer than those above.
total_wikidata_items = query_wiki_db(
    conn=wikidata_db_conn, query=total_wd_items_query, args="1=1"
)

In [47]:
# This is an overestimate from what we see at the top of wikidata.org as that number isn't counting deleted items.
print(f"Total Wikidata items: {total_wikidata_items[0][0]:,}")

Total Wikidata items: 111,351,451
